In [1]:
import torch
source_Info = torch.tensor([
# [x_min, y_min, x_max, y_max, width, height, x_center, y_center]
    [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5],  # 第一组边界框
    [2.0, 2.0, 4.0, 4.0, 2.0, 2.0, 3.0, 3.0],  # 第二组边界框
    [1.0, 1.0, 3.0, 2.0, 2.0, 1.0, 2.0, 1.5],  # 第三组边界框
])


target_Info = torch.tensor([
# [x_min, y_min, x_max, y_max, width, height, x_center, y_center]
    [0.5, 0.5, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0],  # 第一组边界框
    [3.0, 3.0, 5.0, 5.0, 2.0, 2.0, 4.0, 4.0],  # 第二组边界框
    [0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0],  # 第三组边界框
])

d:\anaconda3\envs\2torch1.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
source_area = source_Info[:,4] * source_Info[:,5]
target_area = target_Info[:,4] * target_Info[:,5]
source_area , target_area

(tensor([1., 4., 2.]), tensor([1., 4., 4.]))

In [5]:
torch.max(source_Info[:, :2], target_Info[:, :2])

tensor([[0.5000, 0.5000],
        [3.0000, 3.0000],
        [1.0000, 1.0000]])

In [7]:
torch.min(source_Info[:, 2:4], target_Info[:, 2:4])

tensor([[1., 1.],
        [4., 4.],
        [2., 2.]])

In [8]:
# union_area = 
lt = torch.max(source_Info[:, :2], target_Info[:, :2])  # [M,2]
rb = torch.min(source_Info[:, 2:4], target_Info[:, 2:4])  # [M,2]
wh = torch.clamp(rb - lt, min=0) 
wh 

tensor([[0.5000, 0.5000],
        [1.0000, 1.0000],
        [1.0000, 1.0000]])

In [9]:
union_area = wh[:, 0] * wh[:, 1]  
union_area

tensor([0.2500, 1.0000, 1.0000])

In [12]:
iou = union_area / (source_area + target_area - union_area)
iou

tensor([0.1429, 0.1429, 0.2000])

In [13]:
convex_lt = torch.min(source_Info[:,:2],target_Info[:,:2])
convex_rb = torch.max(source_Info[:,2:4],target_Info[:,2:4])
convex_lt,convex_rb

(tensor([[0., 0.],
         [2., 2.],
         [0., 0.]]),
 tensor([[1.5000, 1.5000],
         [5.0000, 5.0000],
         [3.0000, 2.0000]]))

In [ ]:
# cal the box's area of boxes1 and boxess
boxes1Area = source_Info[:, 4] * source_Info[:, 5]
boxes2Area = target_Info[:, 4] * target_Info[:, 5]
lt = torch.max(source_Info[:, :2], target_Info[:, :2])  # [M,2]
rb = torch.min(source_Info[:, 2:], target_Info[:, 2:])  # [M,2]
wh = torch.clamp(rb - lt, min=0)  # [M,2]
inter_area = wh[:, 0] * wh[:, 1]  # [M]

union_area = boxes1Area + boxes2Area - inter_area
iou = inter_area / union_area
# if iou_type == 'iou':
#     return iou
# w1, h1 = source_Info[:, 4:6]
# w2, h2 = target_Info[:, 4:6]
# center1 = source_Info[:, 6:8]
# center2 = target_Info[:, 6:8]
# out_max_xy = torch.max(source_Info[:, 2:], target_Info[:, 2:])
# out_min_xy = torch.min(source_Info[:, :2], target_Info[:, :2])# [M, 2]
# outer = torch.clamp((out_max_xy - out_min_xy), min=0)  # converx bbox 
# c_area = outer[:, 0] * outer[:, 1] + 1e-16       
# if iou_type == 'giou':
#     return iou - (c_area - union_area) / c_area
# # if DIOU or CIOU:
# outer_diag = (outer[:, 0] ** 2) + (outer[:, 1] ** 2)  # convex diagonal squard length
# inter_diag = (center1[:, 0] - center2[:, 0]) ** 2 + (center1[:, 1] - center2[:, 1]) ** 2
#     # if DIOU:
# if iou_type == 'diou':
#     return iou - inter_diag / outer_diag
#     # elif CIOU:
# if iou_type == 'ciou':
#     arctan = torch.atan(w2 / h2)[:] - torch.atan(w1 / h1)
#     v = (4 / (torch.pi ** 2)) * torch.pow(arctan, 2)
#     S = 1 - iou
#     with torch.no_grad():
#         alpha = v / (S + v)
#         u = inter_diag / outer_diag
#         ciou = iou - (u + v * alpha)
#         ciou = torch.clamp(ciou, min=-1.0, max=1.0)
#     return ciou